# How to store transactions for Azure OpenAI Embedding Model in PromptSail

First you need to get or create an Azure OpenAI embedding deployment. 
You can do it here: https://portal.azure.com/#view/Microsoft_Azure_ProjectOxford/CognitiveServicesHub/~/OpenAI

Click on the chosen resource and than the "Develop" button in the middle of the screen.

Copy the API key and endpoint and paste it into `.env` file in the `examples` folder. 

In [17]:
import os
from dotenv import load_dotenv, dotenv_values
from pprint import pprint


config = dotenv_values(".env")


azure_oai_key = config["AZURE_OPENAI_API_KEY"]
api_base_url = config["AZURE_OPENAI_ENDPOINT"]

api_version = config["AZURE_OPENAI_API_VERSION"]
deployment_name = config["AZURE_EMBEDDING_DEPLOYMENT_NAME"]

print(
    f"Azure OpenAI api key={azure_oai_key[0:3]}...{azure_oai_key[-3:]}"
)
print(
    f"Azure OpenAI api endpoint={api_base_url[0:17]}...{api_base_url[-18:]}"
)
print(f"Azure OpenAI deployment name={deployment_name}")
print(f"Azure OpenAI api version={api_version}")

Azure OpenAI api key=9b2...7b3
Azure OpenAI api endpoint=https://openai-pr....openai.azure.com/
Azure OpenAI deployment name=ada-embeddings
Azure OpenAI api version=2023-07-01-preview


In [18]:

from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings


# Create an instance of the OpenAIEmbeddings class using Azure OpenAI
embeddings = AzureOpenAIEmbeddings(
    openai_api_key=azure_oai_key,
    azure_deployment=deployment_name,
    api_version=api_version,
    azure_endpoint=api_base_url,
    #model="ada-embeddings",
    chunk_size=1)

# Testing embeddings
txt = "The Ultimate Question of Life, the Universe, and Everything is 42."

# Embed a single document
e = embeddings.embed_query(txt)

print(len(e)) # should be 1536

1536


## Create a request to the AzureOpenAI Embedding API via PromptSail proxy

Run the docker and go to PromptSail UI http://localhost/

Create new project with you `project_slug`or edit existing one.

Add your own Azure OpenAI provider by editing the project settings, this will map the Azure OpenAI endpoint to new proxy prompt sail URL. 

Set the `api base url` to your Azure OpenAI endpoint like
 
'https://**azure_openai_resource**.openai.azure.com/'
 
 and add meaningfull `deployment name`.


In mongo it will create new entry in `ai_providers` array, similar to this one

```bash
{
     ai_providers: [
        {
            deployment_name: 'azure US deployment',
            slug: 'azure-us-deployment',
            api_base: 'https://[azure_openai_resource].openai.azure.com/',
            description: '',
            provider_name: 'Azure OpenAI'
        }
    ],
}
```

In this case we will use the default `project 2` settings:
* with project_slug -> 'project2' 
* deployment_name -> 'azure-us-deployment'
resulting in promptsail proxy url like this: 

**"http://localhost:8000/project2/azure-us-deployment"**


In [19]:
from langchain.embeddings import AzureOpenAIEmbeddings


ps_api_base = "http://localhost:8000/project2/azure-us-deployment"

# different deployment
ps_api_base = "http://localhost:8000/edu-project/ps-us2"



# Create an instance of the OpenAIEmbeddings class using Azure OpenAI
embeddings = AzureOpenAIEmbeddings(
    openai_api_key=azure_oai_key,
    azure_deployment=deployment_name,
    api_version=api_version,
    azure_endpoint=ps_api_base,
    #model="ada-embeddings",
    chunk_size=1)

# Testing embeddings
txt = "The Ultimate Question of Life, the Universe, and Everything is 42."

# Embed a single document
e = embeddings.embed_query(txt)

print(len(e)) # should be 1536

1536
